In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle 

ModuleNotFoundError: No module named 'pandas'

In [ ]:
## load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
##preprocessing the data
# drop the columns that are not required
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
# encode the categorical columns
label_gender = LabelEncoder()
data['Gender'] = label_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
## one hot encoder
one_hot_encoder_geo = OneHotEncoder()
geo_encoder = one_hot_encoder_geo.fit_transform(data[['Geography']]).toarray() 
geo_encoder 
one_hot_encoder_geo.get_feature_names_out(['Geography'])
geo_encoded_df = pd.DataFrame(geo_encoder, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [ ]:
##combine one hot encoded data with original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
##save the encoders and scalers
import pickle
with open('label_gender.pkl', 'wb') as file:
    pickle.dump(label_gender, file)
with open('one_hot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geo, file)

In [ ]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split



# Assuming the dataset is named 'data' and it's already processed
# X (features) and y (target)
X = data.drop('EstimatedSalary', axis=1)  # Features (excluding target column 'EstimatedSalary')
y = data['EstimatedSalary']  # Target (EstimatedSalary)


# Split the data into training and testing sets (80% for training and 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)



# Ensure that the data is in the correct format (optional, but ensures consistency)
X_train = np.array(X_train, dtype=np.float32)  # Convert to numpy array with float32
y_train = np.array(y_train, dtype=np.float32)  # Convert to numpy array with float32
X_test = np.array(X_test, dtype=np.float32)  # Convert to numpy array with float32
y_test = np.array(y_test, dtype=np.float32)  # Convert to numpy array with float32


# Now X_train and y_train can be used to train the model


In [ ]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
X_train

array([[667.,   0.,  34., ...,   0.,   0.,   1.],
       [427.,   1.,  42., ...,   0.,   1.,   0.],
       [535.,   0.,  29., ...,   1.,   0.,   0.],
       ...,
       [738.,   1.,  35., ...,   1.,   0.,   0.],
       [590.,   0.,  38., ...,   0.,   0.,   1.],
       [623.,   0.,  48., ...,   0.,   1.,   0.]], dtype=float32)

In [ ]:
y_train

array([163830.64,  57098.  , 185630.77, ..., 181429.88, 148750.16,
       118855.26], dtype=float32)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train, y_train)
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np



In [ ]:
# Normalize the features (standardize them) - already done before but good to repeat
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Normalize the target variable (EstimatedSalary)
y_scaler = StandardScaler()
y_train = y_scaler.fit_transform(y_train.reshape(-1, 1))  # Reshape for scaling
y_test = y_scaler.transform(y_test.reshape(-1, 1))  # Reshape for scaling



In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Define the ANN model architecture for multi-class classification
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Define the ANN model architecture for binary classification
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer
    Dropout(0.2),  # Dropout layer to prevent overfitting
    Dense(64, activation='relu'),  # Hidden layer
    Dropout(0.2),  # Dropout layer
    Dense(32, activation='relu'),  # Hidden layer
    Dense(1)  # Output layer with no activation (linear regression)
])
# Compile the model for regression
model.compile(optimizer='adam', loss='mean_squared_error')




e:\Desktop\sadia waheed GU1\Gen AI course 4 months\ANN-P2\myenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),  # Reduced learning rate
loss='mean_squared_error',  # Suitable for regression
metrics=['mean_absolute_error'])  # MAE for monitoring performance

In [ ]:
# Define callbacks for early stopping and TensorBoard logging
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1)

In [ ]:
# Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=100,  # Number of epochs to train
                    batch_size=32,  # Batch size
                    callbacks=[early_stopping, tensorboard_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 1.0709 - mean_absolute_error: 0.8875 - val_loss: 1.0067 - val_mean_absolute_error: 0.8643
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 1.0533 - mean_absolute_error: 0.8808 - val_loss: 1.0027 - val_mean_absolute_error: 0.8632
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.0559 - mean_absolute_error: 0.8822 - val_loss: 1.0008 - val_mean_absolute_error: 0.8625
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.0490 - mean_absolute_error: 0.8802 - val_loss: 0.9998 - val_mean_absolute_error: 0.8621
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.0393 - mean_absolute_error: 0.8747 - val_loss: 0.9989 - val_mean_absolute_error: 0.8617
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.0192 - mean_absolute_error: 0.8651 - val_loss: 0.9983 - val_mean_absolute_error: 0.8615
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.0288 - mean_absolute_error: 0.8

In [ ]:
# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9994 - mean_absolute_error: 0.8641
Test Loss: 0.9932453036308289, Test MAE: 0.8598814606666565


In [ ]:
# Predicting the EstimatedSalary for the test set
y_pred = model.predict(X_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [ ]:
model.save('model.h5') 

In [ ]:
## load tensorboard extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit 

Reusing TensorBoard on port 6006 (pid 3904), started 20:43:49 ago. (Use '!kill 3904' to kill it.)

In [ ]:
import sklearn
print(sklearn.__version__)


1.6.1
